In [1]:
import numpy as np
import pandas as pd
import patsy

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV

/Users/pt/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/pt/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Advanced Model Tuning

Week 4 | Day 5

### LEARNING OBJECTIVES
*After this lesson, you will be able to:*
- Explain what gridsearch is and why it's useful in machine learning
- Implement auto-tuning in sklearn to find optimal hyperparams

## Recap

At this point we have learned about a couple of different classification models.


Check: Which models do we now know?

- K-nn
- Logistic Regression

## Recap

And we've seen that each of these models have inputs that can be set upon initialization.  These are the hyper-parameters.  They define the behaviour of the model, and are not determined by the data observed.  We set them a priori.

Check: What are some of these inputs? Take a quick look...
    
[K-NN](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)<br>
[Logistic Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

## Given all the possible inputs, how can we fit the best model?

## Autotune!

<img src='http://pix-media.s3.amazonaws.com/blog/1086/t-pain.jpg' width=400>




It isn't realistic for us to attempt to construct and all the possible combinations of models by hand.

Fortunately, sklearn provides a number of built-in functions, that allow us to search the space of all possible models to find the best one. 

We're going to walk through one example of how to do that now step by step...

## We are going to build a model to predict crime in SF

<br>
<center>
<img src="http://static.businessinsider.com/image/563caed4dd08958b7a8b45b6/image.jpg" width=500></center>

## The features will included day of week, time, and district<br><br>
## The target is the type of crime

## Load our data set 

In [2]:
sf_crime = pd.read_csv('./assets/datasets/sf_crime_train.csv')
sf_crime = sf_crime.dropna()

In [3]:
sf_crime.head(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,5/13/15 23:53,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,5/13/15 23:53,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599


## Data type conversions and transformations

In [4]:
sf_crime['Dates'] = pd.to_datetime(sf_crime['Dates'])
sf_crime_dates = pd.DatetimeIndex(sf_crime['Dates'].values, \
                                  dtype='datetime64[ns]', freq=None)

sf_crime['hour'] = sf_crime_dates.hour
sf_crime['month'] = sf_crime_dates.month
sf_crime['year'] = sf_crime_dates.year

## Let's see what all the listed crimes are

In [5]:
sf_crime['Category'].unique()

array(['WARRANTS', 'OTHER OFFENSES', 'LARCENY/THEFT', 'VEHICLE THEFT',
       'VANDALISM', 'NON-CRIMINAL', 'ROBBERY', 'ASSAULT', 'WEAPON LAWS',
       'BURGLARY', 'SUSPICIOUS OCC', 'DRUNKENNESS',
       'FORGERY/COUNTERFEITING', 'DRUG/NARCOTIC', 'STOLEN PROPERTY',
       'SECONDARY CODES', 'TRESPASS', 'MISSING PERSON', 'FRAUD',
       'KIDNAPPING', 'RUNAWAY', 'DRIVING UNDER THE INFLUENCE',
       'SEX OFFENSES FORCIBLE', 'PROSTITUTION', 'DISORDERLY CONDUCT',
       'ARSON', 'FAMILY OFFENSES', 'LIQUOR LAWS', 'BRIBERY',
       'EMBEZZLEMENT', 'SUICIDE', 'LOITERING', 'SEX OFFENSES NON FORCIBLE',
       'EXTORTION', 'GAMBLING', 'BAD CHECKS'], dtype=object)

## We'll select a subsection of the listed crimes

In [6]:
subset = ['VEHICLE THEFT','BURGLARY','DRUG/NARCOTIC']
sf_crime_sub = sf_crime[sf_crime['Category'].isin(subset)]

In [7]:
sf_crime['Category'].isin(subset)[:3]

0    False
1    False
2    False
Name: Category, dtype: bool

In [8]:
sf_crime_sub.head(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,hour,month,year
6,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423327,37.725138,23,5,2015
7,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DONAHUE ST,-122.371274,37.727564,23,5,2015


## Check the total number of districts

In [9]:
sf_crime_sub['PdDistrict'].unique()

array(['INGLESIDE', 'BAYVIEW', 'PARK', 'CENTRAL', 'MISSION', 'SOUTHERN',
       'NORTHERN', 'RICHMOND', 'TARAVAL', 'TENDERLOIN'], dtype=object)

In [10]:
sf_crime_sub.head(2)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,hour,month,year
6,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423327,37.725138,23,5,2015
7,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DONAHUE ST,-122.371274,37.727564,23,5,2015


In [11]:
sf_crime_sub['PdDistrict'].nunique()

10

## Set up our design matrix and target vector with Patsy

### Patsy allows us to use R-style formulas to do this (and other things)
[Patsy Docs](http://patsy.readthedocs.io/en/latest/)

## For now, we are just using Patsy to create dummy variables for hour, day of week and district

In [12]:
X = patsy.dmatrix('~ C(hour) + C(DayOfWeek) + C(PdDistrict)',\
                  sf_crime_sub)
y = sf_crime_sub['Category'].values

In [13]:
y

array(['VEHICLE THEFT', 'VEHICLE THEFT', 'VEHICLE THEFT', ...,
       'VEHICLE THEFT', 'BURGLARY', 'VEHICLE THEFT'], dtype=object)

## Let's look at our design matrix as a DataFrame

In [14]:
pdf = pd.DataFrame(X, columns=X.design_info.column_names)
pdf['Target'] = y
pdf.head(6)

,Intercept,C(hour)[T.1],C(hour)[T.2],C(hour)[T.3],C(hour)[T.4],C(hour)[T.5],C(hour)[T.6],C(hour)[T.7],C(hour)[T.8],C(hour)[T.9],...,C(PdDistrict)[T.CENTRAL],C(PdDistrict)[T.INGLESIDE],C(PdDistrict)[T.MISSION],C(PdDistrict)[T.NORTHERN],C(PdDistrict)[T.PARK],C(PdDistrict)[T.RICHMOND],C(PdDistrict)[T.SOUTHERN],C(PdDistrict)[T.TARAVAL],C(PdDistrict)[T.TENDERLOIN],Target
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VEHICLE THEFT
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VEHICLE THEFT
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VEHICLE THEFT
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,BURGLARY
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VEHICLE THEFT
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VEHICLE THEFT


## Let's see how many districts are listed in our design matrix 

In [15]:
#how many did we start with?
sf_crime_sub['PdDistrict'].nunique()

10

In [16]:
[x for x in pdf.columns if 'PdDistrict' in x]

['C(PdDistrict)[T.CENTRAL]',
 'C(PdDistrict)[T.INGLESIDE]',
 'C(PdDistrict)[T.MISSION]',
 'C(PdDistrict)[T.NORTHERN]',
 'C(PdDistrict)[T.PARK]',
 'C(PdDistrict)[T.RICHMOND]',
 'C(PdDistrict)[T.SOUTHERN]',
 'C(PdDistrict)[T.TARAVAL]',
 'C(PdDistrict)[T.TENDERLOIN]']

In [17]:
#how many do we now have?
pd.Series([x for x in pdf.columns if 'PdDistrict' in x]).nunique()

9

## And how many hours?

In [18]:
sf_crime_sub['hour'].nunique()

24

In [19]:
pd.Series([x for x in pdf.columns if 'hour' in x]).nunique()

23

## Check: Why is there one less on both?

## Set up our training and testing sets

In [20]:
X_train, X_test, y_train, y_test =\
train_test_split(X, y, test_size=0.33, random_state=77)

## Now let's fit a standard logistic regression model

In [21]:
lr = LogisticRegression()

In [22]:
lr_model = lr.fit(X_train, y_train)

## Make our predictions

In [23]:
lr_ypred = lr_model.predict(X_test)

## Check our misclassifications with a confusion matrix

[Confusion Matrix](http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py)

In [24]:
# actual = index; predicted = columns
lr_cm = confusion_matrix(y_test, lr_ypred, labels=lr.classes_)
lr_cm = pd.DataFrame(lr_cm, columns=lr.classes_, index=lr.classes_)
lr_cm

,BURGLARY,DRUG/NARCOTIC,VEHICLE THEFT
BURGLARY,96,19,134
DRUG/NARCOTIC,45,49,76
VEHICLE THEFT,59,11,236


## Check our precision, recall, and f1

[Precision & Recall](https://www.quora.com/What-is-the-best-way-to-understand-the-terms-precision-and-recall)<br>
[F1](https://en.wikipedia.org/wiki/F1_score)

In [25]:
print classification_report(y_test, lr_ypred, labels=lr.classes_)

               precision    recall  f1-score   support

     BURGLARY       0.48      0.39      0.43       249
DRUG/NARCOTIC       0.62      0.29      0.39       170
VEHICLE THEFT       0.53      0.77      0.63       306

  avg / total       0.53      0.53      0.50       725



## Check the CV Score

In [26]:
cvs1 = cross_val_score(lr, X, y, cv=3)
cvs1

array([ 0.473397  ,  0.51983584,  0.55266758])

In [27]:
cvs1.mean()

0.51530013953612797

## Let's now use a penalized regression - we'll use LASSO (L1)

In [28]:
lr_l1 = LogisticRegression(C=10, penalty='l1')

In [29]:
lr_l1_model = lr_l1.fit(X_train, y_train)

In [30]:
lr_l1_ypred = lr_l1_model.predict(X_test)

## Get the confusion matrix

In [31]:
lr_l1_cm = confusion_matrix(y_test, lr_l1_ypred, labels=lr_l1.classes_)
lr_l1_cm = pd.DataFrame(lr_l1_cm, columns=lr_l1.classes_,\
                        index=lr_l1.classes_)
print lr_l1_cm, "\n"

print lr_cm

               BURGLARY  DRUG/NARCOTIC  VEHICLE THEFT
BURGLARY             95             20            134
DRUG/NARCOTIC        45             51             74
VEHICLE THEFT        55             13            238 

               BURGLARY  DRUG/NARCOTIC  VEHICLE THEFT
BURGLARY             96             19            134
DRUG/NARCOTIC        45             49             76
VEHICLE THEFT        59             11            236


## Get the classification report

In [32]:
print classification_report(y_test, lr_l1_ypred, labels=lr_l1.classes_)

               precision    recall  f1-score   support

     BURGLARY       0.49      0.38      0.43       249
DRUG/NARCOTIC       0.61      0.30      0.40       170
VEHICLE THEFT       0.53      0.78      0.63       306

  avg / total       0.53      0.53      0.51       725



## Get mean cross val score

In [33]:
cvs2 = cross_val_score(lr_l1, X, y, cv=3)

In [34]:
print cvs2.mean()

print cvs1.mean()

0.517117903984
0.515300139536


 ## Looks like a minimal + change with L1 penalty at 10, how about other values?

## We can build a function to test this

In [35]:
def test_penalties(c_val):
    lr_l1 = LogisticRegression(C=c_val, penalty='l1', solver='liblinear')
    cvs = cross_val_score(lr_l1, X, y, cv=3, scoring='accuracy')
    return cvs

In [36]:
# let's test it...
test_cs = pd.Series([.001, .01, .1, 1, 1.5, 2.5, 5, 10, 100])\
.to_frame('c_vals')
score_frame = pd.DataFrame([test_penalties(x) for x in test_cs['c_vals']])\
.mean(axis=1).to_frame('score')

final_scores = pd.concat([test_cs, score_frame], axis=1)
final_scores

,c_vals,score
0,0.001,0.333486
1,0.010,0.440547
2,0.100,0.495249
3,1.000,0.512573
4,1.500,0.518032
5,2.500,0.515756
6,5.000,0.520765
7,10.000,0.517118
8,100.000,0.514840


In [37]:
# and so the best c value...
final_scores['c_vals'][final_scores['score'].idxmax()]

5.0

## That wasn't too bad, but...

But that was only changing one parameter. What if we wanted to try out L2 as well as L1? Or if we had a different algorithm with numerous hyperparameters? It can start to become a hassle to code these from scratch.

Fortunately, sk-learn has a [function](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html) that will do this for us.

In [38]:
# fit model with three folds and lasso regularization
# use Cs=20 to test a grid of 20 distinct parameters
# remeber: Cs describes the inverse of regularization strength
logreg_cv = LogisticRegressionCV(Cs=20, solver='liblinear', cv=3,\
                                 penalty='l1', scoring='accuracy')
cv_model = logreg_cv.fit(X_train, y_train)

## Find best C per class (one v. rest)

In [39]:
print('best C for class:')
best_C = {logreg_cv.classes_[i]:x for i, (x, c) in\
          enumerate(zip(logreg_cv.C_, logreg_cv.classes_))}
print(best_C)

best C for class:
{'BURGLARY': 29.763514416313132, 'VEHICLE THEFT': 0.61584821106602605, 'DRUG/NARCOTIC': 0.088586679041008226}


## Get the classification report for our best model

In [40]:
print classification_report(y_test, logreg_cv.predict(X_test))

               precision    recall  f1-score   support

     BURGLARY       0.48      0.38      0.42       249
DRUG/NARCOTIC       0.69      0.25      0.36       170
VEHICLE THEFT       0.52      0.79      0.63       306

  avg / total       0.54      0.52      0.50       725



## Exercise (30 mins)

Using the data set (pdf):
- Fit two models to predict between "Burglary" and "Drug/Narcotic" crimes
- One model should use an L1 penalty and the other should use an L2 penalty
- Make sure to use train_test_split
- Print out a confusion matrix and a classification report for both models
- Finally, build a third model that uses LogisticRegressionCV (you can assume L2)
- Print out a confusion matrix, classification report and the best value of C

## So LogisticRegressionCV is useful for finding the best penalty, but we had to manually change our penalty from 'l1' to 'l2' to try both...What if there was a better way...?

## Introducing GridSearchCV

[GridSearchCV](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html)

## To start we'll select a model and penalties and some hyperparameters 

Then will pass those to GridSearchCV

In [57]:
logreg = LogisticRegression(solver='liblinear')
C_vals = [0.0001, 0.001, 0.01, 0.1, .15, .25, .275, .33, 0.5, .66, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]
penalties = ['l1','l2']

gs = GridSearchCV(logreg, {'penalty': penalties, 'C': C_vals},\
                  verbose=False, cv=15)
gs.fit(X, y)

GridSearchCV(cv=15, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 0.15, 0.25, 0.275, 0.33, 0.5, 0.66, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=False)

## Now let's find the best parameters

In [58]:
gs.best_params_

{'C': 0.1, 'penalty': 'l2'}

## Use this parameter to fit, predict, and print a classification_report for our X and Y

In [59]:
logreg = LogisticRegression(C=gs.best_params_['C'],\
                            penalty=gs.best_params_['penalty'])
cv_model = logreg.fit(vice_X_train, vice_y_train)

In [60]:
cv_pred = cv_model.predict(vice_X_test)

## Now let's check our stats...

In [61]:
cm3 = confusion_matrix(vice_y_test, cv_pred, labels=logreg.classes_)
cm3 = pd.DataFrame(cm3, columns=logreg.classes_, index=logreg.classes_)

In [62]:
cm3

,BURGLARY,DRUG/NARCOTIC
BURGLARY,227,24
DRUG/NARCOTIC,95,60


In [63]:
print classification_report(vice_y_test, cv_pred,\
                            labels=logreg.classes_)

               precision    recall  f1-score   support

     BURGLARY       0.70      0.90      0.79       251
DRUG/NARCOTIC       0.71      0.39      0.50       155

  avg / total       0.71      0.71      0.68       406



## Hold on, how did we find that?

In [64]:
df = pd.DataFrame(columns=penalties, index=C_vals)
for pen in penalties:
    for c in C_vals:
        logreg = LogisticRegression(C=c, penalty=pen)
        logreg.fit(X,y)
        df.loc[c, pen] = cross_val_score(logreg,X,y, cv=15).mean()

In [65]:
df

,l1,l2
0.0001,0.333484,0.440545
0.0010,0.333484,0.440545
0.0100,0.440545,0.480701
0.1000,0.495301,0.499514
0.1500,0.4981,0.497259
0.2500,0.494028,0.497728
0.2750,0.493559,0.497728
0.3300,0.492683,0.496361
0.5000,0.492702,0.495454
0.6600,0.492714,0.493177


## What if I care a lot about precision, but not really about accuracy?

The default scoring metric is determined by the model (but generally, it's accuracy for classification models).

Similar to cross_val_score, you can pick the scoring metric to use for the tuning.  

<img src="./assets/images/scoring.png" width=900></center>

NB some metrics are only available for binary classification, not multi-class.

## We've looked at tuning classification models.  What about regression?

It works the same way.  You can find the best combination of hyperparameters for a regression model too.  


> Check: what metrics might you use to tune the model?

## Recap: how is this different from just fitting models.

Let's say we have data on houses and we want to predict whether a house will sell for over $200k.  We decide to use logistic regression with regularization.  We select ridge with an alpha of 0.1.  These are typically referred to as the hyperparameters.  We set them even before the model has "seen" data.



Then we fit that model to our training data, and we get coefficients.  These are typically referred to as the model parameters.  We learn them from the data.  

Using these coefficients, we can make predictions in our test set and score our model.  



But how do we know that ridge and 0.1 were the "right" hyperparameters to use (i.e. the ones that optimize our model performance)?



We don't really.  So we can try different combinations of regularization hyperparameters and repeat the process (fit model, calculate coefficients, score model) to see which perform best.

And we can use grid-search to do this for us, if we tell it the space of hyper-paramters to search over.

## Independent Practice (20 mins)

Use GridSearchCV with knn on the iris data set:
- Use train_test_split with a train size of .66
- Set a parameter dictionary with the number of neighbors and at least one other parameter
- Get your best estimator and print out a classification report

In [66]:
from sklearn.datasets import load_iris
iris = load_iris()

## Bonus:
Write a function that will return the better performing (based on accuracy) of a tuned knn and a tuned logistic regression.

As arguments, it should take
- the X and y data
- the two models (knn and logistic regression), and a parameter dictionary for each for the grid search

The function should return the hyper-parameters of the best peforming model and its accuracy.

## Conclusion


- We've seen how to use LogisticCV to fit the best regularization hyperparameter
- We've seen how to use GridsearchCV to find the optimal mix of hyperparameters